In [2]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import time
from datetime import datetime
import xmltodict as xtd
import xml.etree.cElementTree as ET
import zillow
import importlib
import sys

# API Keys
from config import gkey, zkey, zws_id, akey

In [ ]:
target_city = 'atlanta'
params = {
    "address": target_city,
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

res = requests.get(base_url, params=params).json()

lat = res["results"][0]["geometry"]["location"]["lat"]
lng = res["results"][0]["geometry"]["location"]["lng"]
state = res["results"][0]["address_components"][2]["short_name"]

params = {
    "latlng":f"{lat},{lng}",
    "result_type":"postal_code",
    "key":gkey
}

res = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json", params=params).json()
zip_code = res['results'][0]['address_components'][0]['long_name']

print(f"city:   {target_city}\nlatlng: {lat}, {lng}\nstate:  {state}\nzip:    {zip_code}")

In [ ]:
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)

In [ ]:
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
loading = '..'
results = []

sys.stdout.write('getting properties..')

for z in zipcodes.ziplist:
    params = {
        "postalcode": z,
        "propertytype":"RESIDENTIAL (NEC)",
        "pagesize":"100"
    }

    headers = {
        "apikey":akey,
        "accept":"application/json"
    }

    res = requests.get(url, headers=headers, params=params).json()
    
    if(res['status']['msg'] == "SuccessWithoutResult"):
        sys.stdout.write(loading)
        sys.stdout.flush()
        time.sleep(10)
        continue
    else:
        sys.stdout.write(f"found {len(results)+1}")
        results.append(res)
        time.sleep(10)

In [ ]:
properties = []
zws_params = []
zids = []

for p in results:
    for q in p['property']:
        properties.append(q)

props_df = pd.DataFrame(properties)
addressId_df = props_df[['address','identifier']]

for i, j in addressId_df.itertuples(index=False):
    zids.append({"zipcode":i['postal1'], "attomId":j['attomId']})
    
zids

In [ ]:
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/basicprofile?"
zid_results = []

sys.stdout.write("getting home values..")

for z in zids:
    price_params = {
        "attomId": z['attomId']
    }

    price_res = requests.get(url, headers=headers, params=price_params).json()
    sys.stdout.write(loading)
    sys.stdout.flush()
    zid_results.append({"attomId": z['attomId'], "zip_code":z['zipcode'], "market_value":price_res})
    time.sleep(10)

In [ ]:
mkt_values = []

for a in aid_results:
    for p in a['property']:
        mkt_values.append(p['assessment']['market']['mktTtlValue'])

mkt_values

In [ ]:
# api = zillow.ValuationApi()
# home_prices = []

# for i, z in enumerate(zws_params):
#     try:
#         zres = api.GetSearchResults(zws_id, z['address'], z['zipcode'], True)
#         price = zres.get_dict()['zestimate']['amount']
#         home_prices.append({"zipcode":z['zipcode'], "price":price})
#     except zillow.ZillowError as e:
#         print(e.message)

# home_prices

In [ ]:
# api = zillow.ValuationApi()
# # home_prices = []

# # for i, z in enumerate(zws_params):
# #     try:
# zres = api.GetSearchResults(zws_id, zws_params[1]['address'], zws_params[2]['zipcode'], True)
# price = zres.get_dict()['zestimate']['amount']
# print(price)
# #         home_prices.append({"zipcode":z['zipcode'], "price":price})
# #     except zillow.ZillowError as e:
# #         print(e.message)

# # home_prices